In [2]:
import pandas as pd
import numpy as np

In [8]:
# Read the file WITHOUT headers first
od_cause = "any_opioid"
df_raw_all_od_deaths = pd.read_excel("drugoverdosedeaths.xlsx", sheet_name=od_cause, header=None)
df_raw_all_od_deaths

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,"Sex, age, race, and Hispanic origin","Drug overdose deaths involving any opioid\1,\5",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1999,2000,2001,2002,2003,2004,2005,2006,2007,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2018\n(single race)
2,All persons,"Deaths per 100,000 resident population",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"All ages, age adjusted\2",2.9,3,3.3,4.1,4.5,4.7,5.1,5.9,6.1,...,6.8,7.3,7.4,7.9,9,10.4,13.3,14.9,- - -,14.6
4,"All ages, crude",2.9,3,3.3,4.1,4.5,4.7,5,5.9,6.1,...,6.8,7.3,7.4,7.9,9,10.3,13.1,14.6,- - -,14.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,"Black or African American, not Hispanic or Lat...",1.6,1.6,1.6,2,2.1,2.1,2.2,2.5,2.5,...,2.6,2.7,2.9,3.1,3.7,4.1,6.1,7.2,7.7,7.6
59,"American Indian or Alaska Native, not Hispanic...",2.2,1.8,3.2,3.2,4.3,4.4,6,5.1,7.4,...,8.7,9,9.8,9.9,8.9,9.4,11.8,12,10.3,10.8
60,"Asian or Pacific Islander, not Hispanic or Lat...",*,*,*,*,*,*,0.4,0.4,0.3,...,0.5,0.5,0.5,0.4,0.6,0.5,0.8,0.9,0.8,…
61,"Asian, not Hispanic or Latina\3,\4",…,…,…,…,…,…,…,…,…,...,…,…,…,…,…,…,…,…,…,0.6


In [9]:
# create a function to extract data from the spreadsheet
def extract_data(df_raw, 
                 years, 
                 data_label, 
                 start_row, 
                 end_row, 
                 start_col, 
                 end_col, 
                 attr_label, 
                 attr_row, 
                 attr_col):
    # Extract data from column 1, starting at start_row to end_row
    data_groups = df_raw.iloc[start_row:end_row, 0].tolist()
    # remove footnotes, if any
    data_groups = [x.split("\\")[0] for x in data_groups]

    # Extract the numeric matrix (start_row onward, start_col onward)
    values = df_raw.iloc[start_row:end_row, start_col:end_col]

    # Extract Label as a value
    attr = df_raw.iloc[attr_row:attr_row+1, attr_col:attr_col+1]
    attr_val = attr.iloc[0][0]
    # clean ":" from the string, if any
    attr_val = attr_val.replace(":", "")

    # Build the DataFrame
    df = pd.DataFrame(values.values, columns=years)

    # Add data_label as a column
    df.insert(0, data_label, data_groups)

    # Add sex as a column
    df.insert(0, attr_label, attr_val)
    return df

In [11]:
# Extract year headers from row 4 (index 1), starting from column 1:21
years = df_raw_all_od_deaths.iloc[1, 1:21].tolist()
years

[1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018]

In [12]:
# extract age-groups
df_age1 = extract_data(df_raw_all_od_deaths, years, 'age_group', 5, 14, 1, 21, 'sex', 2, 0)
df_age1.head(15)

,sex,age_group,1999,2000,2001,2002,2003,2004,2005,2006,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,All persons,Under 15 years,*,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,- - -
1,All persons,15–24 years,1.6,1.9,2.3,2.9,3.4,3.8,4,4.9,...,5.1,5.5,5.8,5.3,5.7,6.2,7,9.3,9.5,- - -
2,All persons,25–34 years,4.1,4.1,4.4,5.6,6.2,6.6,7.4,9,...,10.7,11.4,12.6,12.8,13.8,16.2,19.4,25.9,29.1,- - -
3,All persons,35–44 years,7.2,7.3,7.8,9.5,9.6,9.7,9.6,10.8,...,11.3,11.6,12.4,12.6,13.3,15.6,18.4,24.1,27.3,- - -
4,All persons,45–54 years,5.4,5.8,6.6,8.4,9.2,9.6,10.7,12.2,...,13.4,13.2,14,14.2,14.8,16.1,17.6,21.2,24.1,- - -
5,All persons,55–64 years,1.5,1.6,2,2.4,2.9,3.4,3.9,4.8,...,7,7.4,7.8,8.3,9.9,11.1,12.4,15.2,17,- - -
6,All persons,65–74 years,0.5,0.5,0.5,0.8,0.9,1,1.2,1.2,...,1.9,1.7,2.1,2.3,2.7,3.3,3.5,4.2,4.9,- - -
7,All persons,75–84 years,0.4,0.3,0.4,0.5,0.5,0.6,0.7,0.8,...,0.9,0.8,0.8,1,0.9,1,1.2,1.2,1.4,- - -
8,All persons,85 years and over,*,*,0.5,0.8,0.7,0.6,1,0.8,...,0.8,1.3,0.9,0.9,1,1.1,0.9,0.9,1.2,- - -


In [13]:
# extract age-groups
df_age2 = extract_data(df_raw_all_od_deaths, years, 'age_group', 17, 26, 1, 21, 'sex', 14, 0)
df_age2.head(15)

,sex,age_group,1999,2000,2001,2002,2003,2004,2005,2006,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Male,Under 15 years,*,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.2,0.1,0.1,0.1,0.1,0.1,0.1,0.1,- - -
1,Male,15–24 years,2.4,2.9,3.6,4.3,5.2,5.9,6,7.5,...,7.7,8,8.6,7.9,8.2,8.9,9.8,13.4,13,- - -
2,Male,25–34 years,6.3,6.2,6.6,8.2,8.9,9.4,10.5,13.1,...,15.6,16.2,17.9,17.8,19.7,23.4,28.2,37.3,41.7,- - -
3,Male,35–44 years,10.9,10.7,11,12.9,13,12.5,12.3,14.2,...,14.8,14.7,15.6,16.3,16.9,20.1,24.5,33.3,38.1,- - -
4,Male,45–54 years,8.2,8.6,9.4,11.4,12.3,12.3,13.6,15.3,...,15.8,15.4,16.3,16.6,17.4,19.2,21.2,26.8,31.5,- - -
5,Male,55–64 years,2,1.9,2.4,3.1,3.6,4.1,4.7,5.6,...,8.3,8.7,9.2,10.1,11.9,13.4,15.3,19.3,22.1,- - -
6,Male,65–74 years,0.7,0.5,0.6,0.8,1,1,1,1.3,...,1.9,1.6,2,2.4,2.8,3.4,4.4,5.3,6.3,- - -
7,Male,75–84 years,0.5,*,0.5,0.5,0.4,0.5,0.6,0.7,...,0.8,0.8,0.8,0.9,1,1,1.2,1.4,1.4,- - -
8,Male,85 years and over,*,*,*,*,*,*,*,*,...,1.3,1.4,*,1,1.5,0.9,0.9,1,1.2,- - -


In [14]:
# extract age-groups
df_age3 = extract_data(df_raw_all_od_deaths, years, 'age_group', 29, 38, 1, 21, 'sex', 26, 0)
df_age3.head(15)

,sex,age_group,1999,2000,2001,2002,2003,2004,2005,2006,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Female,Under 15 years,*,*,*,*,*,0.1,*,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,- - -
1,Female,15–24 years,0.7,0.7,1,1.4,1.5,1.6,1.8,2.1,...,2.5,2.8,2.9,2.7,3,3.3,4.1,4.9,5.7,- - -
2,Female,25–34 years,2,2,2.3,3,3.3,3.6,4.3,4.8,...,5.8,6.6,7.3,7.7,7.7,8.8,10.5,14.1,16.1,- - -
3,Female,35–44 years,3.5,3.8,4.6,6.1,6.2,6.8,6.9,7.4,...,8,8.5,9.3,8.9,9.8,11.2,12.5,15,16.5,- - -
4,Female,45–54 years,2.7,3.1,3.8,5.5,6.2,7,7.8,9.3,...,11.1,11.1,11.7,12,12.2,13.1,14,15.8,16.9,- - -
5,Female,55–64 years,1.1,1.4,1.5,1.8,2.3,2.8,3.2,4,...,5.8,6.2,6.4,6.7,7.9,9,9.8,11.5,12.3,- - -
6,Female,65–74 years,0.4,0.5,0.5,0.9,0.8,1,1.4,1.2,...,1.9,1.8,2.1,2.2,2.6,3.1,2.8,3.3,3.6,- - -
7,Female,75–84 years,0.4,*,0.4,0.4,0.6,0.7,0.7,0.8,...,1,0.8,0.7,1,0.8,1.1,1.1,1.1,1.4,- - -
8,Female,85 years and over,*,*,*,0.6,0.8,*,0.9,0.9,...,0.6,1.2,0.9,0.9,0.8,1.2,0.9,0.8,1.1,- - -


In [15]:
df_age = pd.concat([df_age1, df_age2, df_age3], ignore_index=True)
df_age

,sex,age_group,1999,2000,2001,2002,2003,2004,2005,2006,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,All persons,Under 15 years,*,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,- - -
1,All persons,15–24 years,1.6,1.9,2.3,2.9,3.4,3.8,4,4.9,...,5.1,5.5,5.8,5.3,5.7,6.2,7,9.3,9.5,- - -
2,All persons,25–34 years,4.1,4.1,4.4,5.6,6.2,6.6,7.4,9,...,10.7,11.4,12.6,12.8,13.8,16.2,19.4,25.9,29.1,- - -
3,All persons,35–44 years,7.2,7.3,7.8,9.5,9.6,9.7,9.6,10.8,...,11.3,11.6,12.4,12.6,13.3,15.6,18.4,24.1,27.3,- - -
4,All persons,45–54 years,5.4,5.8,6.6,8.4,9.2,9.6,10.7,12.2,...,13.4,13.2,14,14.2,14.8,16.1,17.6,21.2,24.1,- - -
5,All persons,55–64 years,1.5,1.6,2,2.4,2.9,3.4,3.9,4.8,...,7,7.4,7.8,8.3,9.9,11.1,12.4,15.2,17,- - -
6,All persons,65–74 years,0.5,0.5,0.5,0.8,0.9,1,1.2,1.2,...,1.9,1.7,2.1,2.3,2.7,3.3,3.5,4.2,4.9,- - -
7,All persons,75–84 years,0.4,0.3,0.4,0.5,0.5,0.6,0.7,0.8,...,0.9,0.8,0.8,1,0.9,1,1.2,1.2,1.4,- - -
8,All persons,85 years and over,*,*,0.5,0.8,0.7,0.6,1,0.8,...,0.8,1.3,0.9,0.9,1,1.1,0.9,0.9,1.2,- - -
9,Male,Under 15 years,*,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.2,0.1,0.1,0.1,0.1,0.1,0.1,0.1,- - -


In [16]:
# extract race
df_race1 = extract_data(df_raw_all_od_deaths, years, 'race', 40, 51, 1, 21, 'sex', 39, 0)
df_race1.head(15)

,sex,race,1999,2000,2001,2002,2003,2004,2005,2006,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Male,White,4.4,4.5,5,6.1,6.7,7,7.4,8.7,...,9.9,10.1,10.8,11,11.7,13.4,15.8,20.4,22.7,22
1,Male,Black or African American,5.7,5.6,5.3,5.3,5,4.4,4.8,6,...,4.7,4.1,4.7,5.1,6.3,7.4,9.1,14.7,18.7,20.6
2,Male,American Indian or Alaska Native,3.2,3.4,3,4.4,5.1,6.2,5.7,6.9,...,10,6.3,7.2,7.9,7.2,9.7,9.3,10.2,11.9,11.7
3,Male,Asian or Pacific Islander,*,0.5,0.4,0.8,0.5,0.7,0.9,0.8,...,0.9,0.9,1.3,1.1,1.3,1.4,1.7,2.3,2.4,2.5
4,Male,Hispanic or Latino,5.8,4.5,4.2,4.8,5.1,4.3,4.5,4.9,...,4.8,4.1,4.6,5,5.6,5.7,6.7,9.2,10.5,11.6
5,Male,"White, not Hispanic or Latino",4.1,4.4,5.1,6.4,7,7.6,8,9.6,...,11.1,11.6,12.4,12.5,13.3,15.5,18.1,23.4,26.1,25
6,Male,"Black or African American, not Hispanic or Latino",5.7,5.7,5.4,5.4,5.1,4.5,4.8,6.2,...,4.9,4.3,4.9,5.3,6.5,7.6,9.3,15.1,19.4,21.2
7,Male,"American Indian or Alaska Native, not Hispanic...",3.6,3.7,3.8,5.1,6.3,8.2,7.8,9.1,...,15.2,10,11.7,12.2,11.3,15.2,14.9,16.1,19.5,18.3
8,Male,"Asian or Pacific Islander, not Hispanic or Latino",*,0.5,0.4,0.8,0.5,0.6,0.9,0.7,...,0.9,0.9,1.2,1.2,1.3,1.3,1.7,2.3,2.3,2.4
9,Male,"Asian, not Hispanic or Latino",…,…,…,…,…,…,…,…,...,…,…,…,…,…,…,…,…,…,…


In [17]:
# extract race
df_race2 = extract_data(df_raw_all_od_deaths, years, 'race', 52, 63, 1, 21, 'sex', 51, 0)
df_race2.head(15)

,sex,race,1999,2000,2001,2002,2003,2004,2005,2006,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Female,White,1.5,1.7,2.1,2.8,3.1,3.5,3.8,4.4,...,5.3,5.6,6.1,6.1,6.6,7.4,8.2,9.7,10.7,10.2
1,Female,Black or African American,1.5,1.5,1.6,2,2,2,2.1,2.4,...,2.4,2.5,2.6,2.8,3,3.6,4,5.9,6.9,7.4
2,Female,American Indian or Alaska Native,1.9,1.4,2.5,2.6,3.3,3.3,4.6,3.6,...,5.3,5.7,5.9,6.8,6.5,5.8,6.5,7.6,7.5,6.7
3,Female,Asian or Pacific Islander,*,*,*,*,*,0.3,0.4,0.4,...,0.4,0.5,0.5,0.5,0.4,0.6,0.5,0.8,0.9,0.8
4,Female,Hispanic or Latina,1,0.8,1,1.5,1.3,1.4,1.4,1.6,...,1.6,1.6,1.9,1.9,2.1,2.3,2.4,3,3.1,3.3
5,Female,"White, not Hispanic or Latina",1.6,1.8,2.2,3.1,3.4,3.9,4.3,4.9,...,6.1,6.5,7,7,7.6,8.5,9.6,11.4,12.7,12.1
6,Female,"Black or African American, not Hispanic or Latina",1.6,1.6,1.6,2,2.1,2.1,2.2,2.5,...,2.5,2.6,2.7,2.9,3.1,3.7,4.1,6.1,7.2,7.7
7,Female,"American Indian or Alaska Native, not Hispanic...",2.2,1.8,3.2,3.2,4.3,4.4,6,5.1,...,7.6,8.7,9,9.8,9.9,8.9,9.4,11.8,12,10.3
8,Female,"Asian or Pacific Islander, not Hispanic or Latina",*,*,*,*,*,*,0.4,0.4,...,0.4,0.5,0.5,0.5,0.4,0.6,0.5,0.8,0.9,0.8
9,Female,"Asian, not Hispanic or Latina",…,…,…,…,…,…,…,…,...,…,…,…,…,…,…,…,…,…,…


In [18]:
df_race = pd.concat([df_race1, df_race2], ignore_index=True)
df_race

,sex,race,1999,2000,2001,2002,2003,2004,2005,2006,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Male,White,4.4,4.5,5,6.1,6.7,7,7.4,8.7,...,9.9,10.1,10.8,11,11.7,13.4,15.8,20.4,22.7,22
1,Male,Black or African American,5.7,5.6,5.3,5.3,5,4.4,4.8,6,...,4.7,4.1,4.7,5.1,6.3,7.4,9.1,14.7,18.7,20.6
2,Male,American Indian or Alaska Native,3.2,3.4,3,4.4,5.1,6.2,5.7,6.9,...,10,6.3,7.2,7.9,7.2,9.7,9.3,10.2,11.9,11.7
3,Male,Asian or Pacific Islander,*,0.5,0.4,0.8,0.5,0.7,0.9,0.8,...,0.9,0.9,1.3,1.1,1.3,1.4,1.7,2.3,2.4,2.5
4,Male,Hispanic or Latino,5.8,4.5,4.2,4.8,5.1,4.3,4.5,4.9,...,4.8,4.1,4.6,5,5.6,5.7,6.7,9.2,10.5,11.6
5,Male,"White, not Hispanic or Latino",4.1,4.4,5.1,6.4,7,7.6,8,9.6,...,11.1,11.6,12.4,12.5,13.3,15.5,18.1,23.4,26.1,25
6,Male,"Black or African American, not Hispanic or Latino",5.7,5.7,5.4,5.4,5.1,4.5,4.8,6.2,...,4.9,4.3,4.9,5.3,6.5,7.6,9.3,15.1,19.4,21.2
7,Male,"American Indian or Alaska Native, not Hispanic...",3.6,3.7,3.8,5.1,6.3,8.2,7.8,9.1,...,15.2,10,11.7,12.2,11.3,15.2,14.9,16.1,19.5,18.3
8,Male,"Asian or Pacific Islander, not Hispanic or Latino",*,0.5,0.4,0.8,0.5,0.6,0.9,0.7,...,0.9,0.9,1.2,1.2,1.3,1.3,1.7,2.3,2.3,2.4
9,Male,"Asian, not Hispanic or Latino",…,…,…,…,…,…,…,…,...,…,…,…,…,…,…,…,…,…,…


In [19]:
pd.set_option('future.no_silent_downcasting', True)

In [20]:
def convert_to_tables(df_age, df_race):
    # Replace ellipses with NaN
    df_age = df_age.replace("…", np.nan)

    # Replace ellipses with NaN
    df_race = df_race.replace("…", np.nan)

    # Melt the year columns into a table
    table_age = df_age.melt(
        id_vars=["sex", "age_group"],
        var_name="year",
        value_name="rate"
    )

    # Melt the year columns into a table
    table_race = df_race.melt(
        id_vars=["sex", "race"],
        var_name="year",
        value_name="rate"
    )

    # Convert types
    table_age["year"] = pd.to_numeric(table_age["year"], errors="coerce")
    table_age["rate"] = pd.to_numeric(table_age["rate"], errors="coerce")

    # Convert types
    table_race["year"] = pd.to_numeric(table_race["year"], errors="coerce")
    table_race["rate"] = pd.to_numeric(table_race["rate"], errors="coerce")

    return [table_age, table_race]

In [21]:
[t_age, t_race] = convert_to_tables(df_age, df_race)

In [22]:
t_age

,sex,age_group,year,rate
0,All persons,Under 15 years,1999,NaN
1,All persons,15–24 years,1999,1.6
2,All persons,25–34 years,1999,4.1
3,All persons,35–44 years,1999,7.2
4,All persons,45–54 years,1999,5.4
...,...,...,...,...
535,Female,45–54 years,2018,NaN
536,Female,55–64 years,2018,NaN
537,Female,65–74 years,2018,NaN
538,Female,75–84 years,2018,NaN


In [23]:
t_race

,sex,race,year,rate
0,Male,White,1999,4.4
1,Male,Black or African American,1999,5.7
2,Male,American Indian or Alaska Native,1999,3.2
3,Male,Asian or Pacific Islander,1999,NaN
4,Male,Hispanic or Latino,1999,5.8
...,...,...,...,...
435,Female,"Black or African American, not Hispanic or Latina",2018,7.7
436,Female,"American Indian or Alaska Native, not Hispanic...",2018,10.3
437,Female,"Asian or Pacific Islander, not Hispanic or Latina",2018,0.8
438,Female,"Asian, not Hispanic or Latina",2018,NaN
